In [1]:
import numpy as np
import pandas as pd 
from elasticsearch import Elasticsearch
import sklearn.metrics as sl


In [2]:
from elasticsearch import helpers
client = Elasticsearch(
    "http://localhost:9200",
     basic_auth=("elastic", "elastic")

)
resp = client.info()

In [18]:
import os
for dirname, _, filenames in os.walk('/data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        with open(os.path.join(dirname, filename)) as f:
            line_count = 0
            id_set = set()
            for l in f.readlines():
                line_count += 1
                if filename == "CISI.REL":
                    id_set.add(l.lstrip(" ").split(" ")[0])
                elif l.startswith(".I "):
                    id_set.add(l.split(" ")[1].strip())
            print(f"{filename} : {len(id_set)} items, over {line_count} lines.")

In [3]:
temp_id = 0
temp_doc_id = []
rel = {}
with open('data\CISI.REL') as f:
    lines = ""
    for l in f.readlines():
        qid = l.strip().split()[0]
        if(qid == temp_id):
            temp_doc_id.append(l.strip().split()[1])
        else:
            rel[temp_id] = temp_doc_id
            temp_id = qid
            temp_doc_id = []
            temp_doc_id.append(l.strip().split()[1])
rel.pop(0) 
print(rel['3'])

['60', '85', '114', '123', '126', '131', '133', '136', '138', '140', '346', '359', '363', '372', '412', '445', '454', '461', '463', '469', '532', '537', '540', '553', '554', '555', '585', '590', '599', '640', '660', '664', '803', '901', '909', '911', '1027', '1053', '1169', '1179', '1181', '1190', '1191', '1326']


In [9]:
with open('data\CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    

print(len(lines))

7475


In [6]:
qry_list =[]
qry_id = ""
qry_text = ""
with open('data\CISI.QRY') as f:
    qry = ""
    for l in f.readlines():
        qry += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    qry = qry.lstrip("\n").split("\n")
for l in qry:
    if l.startswith(".I"):
        qry_list.append({'id':qry_id, 'doc':qry_text.lstrip(" ")})
        qry_id = l.split(" ")[1].strip()
        qry_text = ""
    
    else:
        qry_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.


qry_list.remove({'id':'' ,'doc':''})


In [10]:
doc_set = {}
doc_list = []
doc_id = ""
doc_text = ""
ind = 1
doc_df = pd.DataFrame(columns=['id','doc'])
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_df.loc[ind] = [doc_id, doc_text.lstrip(" ")]
        ind = ind + 1
       # doc_df.append({'id':doc_id,'doc':doc_text.lstrip(" ")}, ignore_index=True)
       # doc_df['doc'] = doc_text.lstrip(" ")
        doc_set['doc'] = doc_text.lstrip(" ")
        doc_set['id'] = doc_id
        doc_list.append({'id':doc_id, 'doc':doc_text.lstrip(" ")})
        doc_set['doc'] = ""
        doc_set['id'] = ""
        doc_id = ""
        doc_text = ""
    
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

# Print something to see the dictionary structure, etc.
print(f"Number of documents = {len(doc_list)}" + ".\n")


Number of documents = 1460.



In [11]:
i = 0
for rec in doc_list:
    i = i + 1
    client.index(index="anacisi", id = i, document=rec)

In [ ]:
all = client.search(index="cisi", query={"match_all": {}})
print("Got %d Hits:" % all['hits']['total']['value'])
for hit in all['hits']['hits']:
    print("%(doc)s : %(id)s" % hit["_source"])

In [81]:

truth =[]
predect = []
for i in range(2,3):
    print(qry_list[i]['doc'] )
    query = {
                "match": {
                    "doc": qry_list[i]['doc'] 
                }
            }
    ids =[]
    docs =[]
    resp = client.search(index="cisi", query=query, size=100)
    print("Got %d Hits" % resp['hits']['total']['value'])
    for hit in resp['hits']['hits']:
        # print("%(id)s " % hit["_source"])
        ids.append(hit['_source']['id'])
        docs.append(hit['_source']['doc'])
    predect.append(ids)
  
    truth.append(rel[str(i+1)])

What is information science?  Give definitions where possible. 
Got 1311 Hits


In [1]:
sl.precision_score(truth,predect,average='micro')

NameError: name 'sl' is not defined

In [72]:
def precision(true:list, predect:list):
    f = 0
    for t in true :
        for p in predect:
            if p == t:
                f = f + 1
    return f /len(predect)


In [73]:
def recall(true:list, predect:list):
    f = 0
    for t in true :
        for p in predect:
            if p == t:
                f = f + 1
    return f /len(true)

In [82]:
print(precision(truth[0],predect[0]))
print(recall(truth[0],predect[0]))

0.14
0.3181818181818182


In [1]:
class QueryProcessor:

    def __init__(self, nlp, keep=None):
        self.nlp = nlp
        self.keep = keep or {'PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'}

    def generate_query(self, text):
        doc = self.nlp(text)
        query = ' '.join(token.text for token in doc if token.pos_ in self.keep)
        return query


In [8]:
qp = QueryProcessor()

TypeError: __init__() missing 1 required positional argument: 'nlp'